In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import glob
import os
import csv
import scipy.io
from PIL import Image
%matplotlib inline

In [2]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  return tf.train.Feature(bytes_list=tf.train.FloatList(value=[value]))

Read the pictures

In [10]:
Data_PATH = "./img/cam1/"
def read_images_from(path=Data_PATH):
  images = []
  jpg_files_path = glob.glob(os.path.join(path, '*.[jJ][pP][gG]'))
  for filename in jpg_files_path:
    im = Image.open(filename).convert("L")  # Convert to greyscale
    im = np.asarray(im, np.uint8)
    # print(type(im))
    # get only images name, not path
    image_name = filename.split('/')[-1].split('.')[0][5:]
    images.append([int(image_name), im])

  images = sorted(images, key=lambda image: image[0])

  images_only = [np.asarray(image[1], np.uint8) for image in images]  # Use unint8 or you will be !!!
  images_only = np.array(images_only)

  #print(images_only.shape)
  return images_only

def read_labels_from(path='./'):
  with open(os.path.join(path, 'labels.csv'), 'r') as dest_f:
    data_iter = csv.reader(dest_f)
    train_labels = [data for data in data_iter]

  # pre process labels to int
  train_labels = train_labels[1:]
  train_labels = np.array(train_labels, dtype=np.float64)

  return train_labels

In [11]:
labels = read_labels_from()
labels = np.concatenate((labels[:9], labels[10:]))
labels = np.repeat(labels, 158, axis=0)

In [12]:
cameras = list(range(1,9))
cameras.extend(list(range(10,19)))
images = []
for camera in cameras:
    #print('./img/cam' + str(camera) + '/')
    images.append(read_images_from('./img/cam' + str(camera) + '/'))
ans = np.concatenate(images)

In [9]:
for i in range(len(images)):
    print(images[i].shape)

(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)
(158, 480, 640)
(158, 60, 80)
(158, 60, 80)
(158, 60, 80)


In [13]:
images = ans
file_dict = {}
file_dict['images'] = images
file_dict['labels'] = labels
scipy.io.savemat('gazebo_data.mat', file_dict)

In [ ]:
def convert_to(images, labels, name):
  print('labels shape is ', labels.shape[0])
  rows = images.shape[1]
  cols = images.shape[2]
  num_examples = labels.shape[0]
  filename = os.path.join('./gazebo_data.tfrecords.tf')
  print('Writing', filename)
  writer = tf.python_io.TFRecordWriter(filename)
  for index in range(num_examples):
    image_raw = images[index, :, :].tostring()
    example = tf.train.Example(features=tf.train.Features(feature={
        'height': _int64_feature(rows),
        'width': _int64_feature(cols),
        'label': _float_feature(labels[index, :]),
        'image_raw': _bytes_feature(image_raw)}))
    writer.write(example.SerializeToString())

In [18]:
convert_to(images, labels, 'train')


labels shape is  2686
Writing ./gazebo_data.tfrecords.tf


TypeError: array([ 3.,  4.,  1.,  0.,  3.,  1.]) has type <class 'numpy.ndarray'>, but expected one of: (<class 'float'>, <class 'int'>, <class 'int'>)

In [60]:
tf.train.Feature

tensorflow.core.example.feature_pb2.Feature